# Aula 01 - DASK

## Instalandos as bibliotecas

In [5]:
%%bash
if command -v uv > /dev/null; then
    uv add --upgrade "dask[dataframe]"
    uv add --upgrade "dask[distributed]"
else
    pip install --upgrade "dask[dataframe]"
    pip install --upgrade "dask[distributed]"
fi

Resolved 50 packages in 321ms
Audited 41 packages in 0.04ms
Resolved 58 packages in 432ms
Prepared 8 packages in 144ms
Installed 8 packages in 3ms
 + distributed==2025.2.0
 + jinja2==3.1.5
 + markupsafe==3.0.2
 + msgpack==1.1.0
 + sortedcontainers==2.4.0
 + tblib==3.0.0
 + urllib3==2.3.0
 + zict==3.0.0


## Provisionando estrutura para a aplicação

### Realizando os imports

In [ ]:
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster

### Criando um cluster local e um cliente

In [3]:
# Criar cluster local anexado ao kernel do notebook
cluster = LocalCluster()
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 16,Total memory: 31.09 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:45931,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 31.09 GiB
Comm: tcp://127.0.0.1:41277,Total threads: 4
Dashboard: http://127.0.0.1:37841/status,Memory: 7.77 GiB
Nanny: tcp://127.0.0.1:46039,


## Aplicação

In [7]:
BIDFILE_PATH = "../../data/bigfile-001.csv"

In [8]:
%%time

# Criar variável que aponta para o arquivo de 2.8GB
dd_df = dd.read_csv(BIDFILE_PATH, header=None)
type(dd_df)

CPU times: user 14 ms, sys: 1.14 ms, total: 15.1 ms
Wall time: 14 ms


dask.dataframe.dask_expr._collection.DataFrame

In [9]:
dd_df

,0,1,2,3,4
npartitions=47,,,,,
,int64,string,string,string,string
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [10]:
%%time

# Dar nome as colunas do arquivo de 2.8GB
dd_df.columns = ['Height', 'Input', 'Output', 'Sum', 'Time']
dd_df

CPU times: user 134 μs, sys: 22 μs, total: 156 μs
Wall time: 160 μs


,Height,Input,Output,Sum,Time
npartitions=47,,,,,
,int64,string,string,string,string
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [11]:
%%time

len(dd_df)

CPU times: user 415 ms, sys: 57.6 ms, total: 472 ms
Wall time: 6.97 s


13494203

In [12]:
%%time

# Ler as primeiras 5 linhas do arquivo bigfile.csv
pd_df = dd_df.head()
type(pd_df)

CPU times: user 25.7 ms, sys: 2.4 ms, total: 28.1 ms
Wall time: 507 ms


pandas.core.frame.DataFrame

In [13]:
pd_df

,Height,Input,Output,Sum,Time
0,546,"['1DZTzaBHUDM7T3QvUKBz4qXMRpkg8jsfB5', '1DCbY2...","['1KAD5EnzzLtrSo2Da2G4zzD7uZrjk8zRAv', '1', '1...",['25'],2009-01-15 06:08:20
1,546,"['1KAD5EnzzLtrSo2Da2G4zzD7uZrjk8zRAv', '1DZTza...","['1KAD5EnzzLtrSo2Da2G4zzD7uZrjk8zRAv', '25']",['25'],2009-01-15 06:08:20
2,546,['1KAD5EnzzLtrSo2Da2G4zzD7uZrjk8zRAv'],"['1DZTzaBHUDM7T3QvUKBz4qXMRpkg8jsfB5', '1', '1...",['25'],2009-01-15 06:08:20
3,586,"['1DNdPgBZRWjDj1JbVZQEYMv7jvqJF7R4Py', '1ELmSk...","['19QKDUJtx9n7Vaga6nX1bVHdsnT4Khfyi6', '250']",['250'],2009-01-15 14:25:20
4,586,['1E5npMTmh9PnBHxCacuqzAZRGEybkxPMAq'],"['1QKJnFPvRoZ7avBhxqwxhsEv62JWYoB1R9', '50']",['50'],2009-01-15 14:25:20


In [14]:
%%time

# Criar uma variável que aponta para a operação soma de todos os números
# da coluna 'Height' do tipo int
sum_df = dd_df['Height'].sum()
type(sum_df)

CPU times: user 858 μs, sys: 0 ns, total: 858 μs
Wall time: 862 μs


dask.dataframe.dask_expr._collection.Scalar

In [15]:
%%time

# Calcular de forma distribuída a soma de todos os números da coluna 'Height'
# do tipo int
result_sum = sum_df.compute()
type(result_sum)

CPU times: user 427 ms, sys: 67.6 ms, total: 495 ms
Wall time: 8.4 s


numpy.int64

In [ ]:
result_sum

np.int64(2589395531434)

In [17]:
# Criar uma variável que aponta para a operação média de todos os números
# da coluna 'Height' do tipo int
mean_df = dd_df['Height'].mean()
type(mean_df)

dask.dataframe.dask_expr._collection.Scalar

In [18]:
%%time

# Calcular de forma distribuída a média de todos os números da coluna 'Height'
# do tipo int
result_mean = mean_df.compute()
type(result_mean)

CPU times: user 538 ms, sys: 54.2 ms, total: 592 ms
Wall time: 8.83 s


numpy.float64

In [23]:
result_mean

np.float64(191889.4751645577)

## Liberando recursos

In [21]:
# Finalizar client e cluster para liberar recursos alocados
client.close()
cluster.close()